## Урок 5. #Задача оттока: варианты постановки, возможные способы решения

### Вопросы

#### Вопрос 1: объясните своими словами смысл метрик Precison, Recall *
1. Какова их взаимосвязь и как с ними связан порог вероятности? 
2. Можно ли подобрать порог так, что recall будет равен 1? Что при этом будет с precision
3. Аналогичный вопрос про precision

Precision отражает точность определения целевого класса, т. е. долю правильно предсказанных объектов.
Recall характеризует полноту охвата объектов целевого класса, т. е. долю объектов целевого класса от общего количества объектов.

Они, как правило, имеют обратную зависимость, т. к. с ростом порога будет расти точность, а с уменьшением - полнота. 

Чтобы recall был равен 1, FN должен быть равен 0, а порого должен минимизироваться. Такое возможно в случает отсутствия ошибки 2-го рода - все объекты целевого класса должны быть правильно предсказаны. 
Возможны следующие варианты: 
- идеальная модель;
- дисбаланс классов (Отсутствие объектов целевого класса в тестовой выборке); 
- модель, присваивающая всем объектам метку целевого класса.

Чтобы precision был равен 1, FP должен быть равен 0, а порог должен максимизироваться. Такое возможно в случает отсутствия ошибки 1-го рода - все предсказанные объекты целевого класса действительно должны относиться к целевому классу. 
Возможны следующие варианты: 
- идеальная модель;
- дисбаланс классов (все объекты в тестовой выборке относятся к целевому классу).

Стоит отметить, что в случае с recall мы можем искуственно создать условия, при которых данная метрика будет равна 1 (порог = 0). В случае precision, задав порог = 1, мы должны иметь модель хорошо разделяющая классы объектов.


<b>Вопрос 2: предположим, что на удержание одного пользователя у нас уйдет 1 доллар. При этом средняя ожидаемая прибыль с каждого TP (true positive) - 2 доллара. Оцените качество модели выше с учетом этих данных и ответьте на вопрос, является ли она потенциально экономически целесообразной?</b>

In [115]:
TP, FP = 1832, 159

profit = TP * 2 - (TP + FP) * 1
profit

1673

### Задания

In [116]:
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin 

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [117]:
df = pd.read_csv("churn_data.csv")

In [118]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)


In [119]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [120]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [121]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
        ('selector', FeatureSelector(column=cat_col)),
        ('ohe', OHEEncoder(key=cat_col))
    ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('standard', StandardScaler())
    ])
    final_transformers.append((cont_col, cont_transformer))

In [122]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [123]:
classifiers = {
    'RandomForestClassifier': Pipeline([
        ('features', feats),
        ('classifier', RandomForestClassifier(random_state = 42))]),
    'LogisticRegression': Pipeline([
        ('features', feats),
        ('classifier', LogisticRegression(random_state = 42))]),
    'CatBoostClassifier': Pipeline([
        ('features', feats),
        ('classifier', CatBoostClassifier(random_state = 42, silent=True))])
    }

In [124]:
# Сформируем структуру таблицы со статистикой

valid_cols = ['mean roc auc', 'roc auc std']

test_cols = ['threshold',
        'precision',
        'recall',
        'fscore',
        'roc auc',
        'profit']

cols_names = [('validation', c) for c in valid_cols] + [('test', c) for c in test_cols]

multi_cols = pd.MultiIndex.from_tuples(cols_names)
stat_df = pd.DataFrame(columns=multi_cols, index=classifiers.keys())

In [125]:
for (name, classifier) in classifiers.items():
    # запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    # обучим пайплайн на всем тренировочном датасете и предскажем значения тестовой выборки
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)    
    roc_auc = roc_auc_score(y_test.values, y_score)
    
    cnf_matrix = confusion_matrix(y_test, y_score > thresholds[ix])
    TP, FP = cnf_matrix[0][:]
    profit = TP * 2 - (TP + FP) * 1
    
    stat_df.at[name, ('validation', 'mean roc auc')] = cv_score
    stat_df.at[name, ('validation', 'roc auc std')] = cv_score_std
    stat_df.at[name, ('test', 'threshold')] = thresholds[ix]
    stat_df.at[name, ('test', 'precision')] = precision[ix]
    stat_df.at[name, ('test', 'recall')] = recall[ix]
    stat_df.at[name, ('test', 'fscore')] = fscore[ix]
    stat_df.at[name, ('test', 'roc auc')] = roc_auc  
    stat_df.at[name, ('test', 'profit')] = profit   
    
stat_df

validation                  test                      \
                       mean roc auc roc auc std threshold precision    recall   
RandomForestClassifier      0.84698   0.0208367      0.38  0.654397  0.628684   
LogisticRegression         0.761725   0.0210729  0.289522    0.4624   0.56778   
CatBoostClassifier         0.861037   0.0178314  0.386362  0.661191  0.632613   

                                                   
                          fscore   roc auc profit  
RandomForestClassifier  0.641283  0.863699   1675  
LogisticRegression        0.5097  0.772077   1319  
CatBoostClassifier      0.646586  0.876942   1661

**Вывод: наиболее подходящей метрикой в данном случае будет являться f-score, так как нам нужно добиться максимизации прибыли, что складывается из максимального значения TP и минимального значения FP (см. вопрос 2). Лучше себя показала модель градиентного бустинга на основе CatBoost.**

### Подбор гиперпараметров модели CatBoost

In [137]:
# Создаем таблицу со статистикой

stat_df_cat_boost = pd.DataFrame(
    data=None, 
    columns=stat_df.columns)

In [138]:
# Инициализируем наборы параметров и pipelines с классификаторами
# (по-хорошему нужно сделать одинаковую предообработку)

trees = [10, 20, 50, 100, 500]
depths = [2, 3, 5, 10, 15]
weights = [[1, 1], [1, 2], [1, 3], [1, 3.5]]

params = {'n_estimators':[10, 20, 50, 100, 500],
          'max_depth':[2, 3, 5, 10, 15],
          'class_weights': [[1, 1], [1, 2], [1, 3], [1, 3.5]]}

classifiers = {}

for n_tree in trees:
    for depth in depths:
        for weight in weights:

classifiers = {
    'RandomForestClassifier': Pipeline([
        ('features', feats),
        ('classifier', RandomForestClassifier(random_state = 42))]),
    'LogisticRegression': Pipeline([
        ('features', feats),
        ('classifier', LogisticRegression(random_state = 42))]),
    'CatBoostClassifier': Pipeline([
        ('features', feats),
        ('classifier', CatBoostClassifier(random_state = 42, silent=True))])
    }

n_estimators [10, 20, 50, 100, 500]
max_depth [2, 3, 5, 10, 15]
class_weights [[1, 1], [1, 2], [1, 3], [1, 3.5]]


In [136]:
%%time
print()


Wall time: 0 ns


In [133]:
stat_df_cat_boost = pd.DataFrame(
    data=None, 
    columns=stat_df.columns)

In [134]:
stat_df_cat_boost

Empty DataFrame
Columns: [(validation, mean roc auc), (validation, roc auc std), (test, threshold), (test, precision), (test, recall), (test, fscore), (test, roc auc), (test, profit)]
Index: []